# Data Collection

## Asset Selection: 
Category	Tickers
### Large-cap stocks	
    AAPL, MSFT, AMZN, JNJ, NVDA
### Mid/small-cap	
    PLTR, ROKU, ETSY, UPST, SOFI
### ETFs	
    QQQ, ARKK, SPY
### Indexes	
    GSPC (S&P 500), IXIC (NASDAQ)

#### Time Range: 2019-01-01 to 2024-12-31

In [82]:
import yfinance as yf
import pandas as pd
import numpy as np

In [83]:
tickers = {
    "Large-cap stocks": ["AAPL", "MSFT", "AMZN", "JNJ", "NVDA"],
    "Mid/small-cap": ["PLTR", "ROKU", "ETSY", "UPST", "SOFI"],
    "ETFs": ["QQQ", "ARKK", "SPY"],
    "Indexes": ["^GSPC", "^IXIC"]
}

In [84]:
all_tickers = [ticker for group in tickers.values() for ticker in group]

In [85]:
start_date = "2019-01-01"
end_date = "2024-12-31"
data = yf.download(all_tickers, start=start_date, end=end_date, group_by='ticker')

[*********************100%***********************]  15 of 15 completed


### Calculating Daily Returns, Log Returns abd Volatility of Window size: 20 Days

In [86]:
frames = []
for ticker in all_tickers:
    df = data[ticker].copy()
    df["Ticker"] = ticker
    df["Date"] = df.index
    df = df.reset_index(drop=True)
    df.dropna(inplace = True)
    df["Daily Return"] = df["Close"].pct_change()
    df["Log Return"] = np.log1p(df["Daily Return"])
    df["MA_20"] = df["Close"].rolling(window=20).mean()
    df["Volatility_20d"] = df["Daily Return"].rolling(window=20).std()
    frames.append(df)

### Checking and Filling for null values 

In [87]:
df.isnull().sum()

Price
Open               0
High               0
Low                0
Close              0
Volume             0
Ticker             0
Date               0
Daily Return       1
Log Return         1
MA_20             19
Volatility_20d    20
dtype: int64

In [90]:
df["Ticker"].unique

<bound method Series.unique of 0       ^IXIC
1       ^IXIC
2       ^IXIC
3       ^IXIC
4       ^IXIC
        ...  
1504    ^IXIC
1505    ^IXIC
1506    ^IXIC
1507    ^IXIC
1508    ^IXIC
Name: Ticker, Length: 1509, dtype: object>

In [91]:
merged_df = pd.concat(frames, ignore_index=True)

In [92]:
merged_df.isnull().sum()

Price
Open                0
High                0
Low                 0
Close               0
Volume              0
Ticker              0
Date                0
Daily Return       15
Log Return         15
MA_20             285
Volatility_20d    300
dtype: int64

In [95]:
merged_df['Daily Return'] = merged_df['Daily Return'].fillna(0)
merged_df['Log Return'] = merged_df['Log Return'].fillna(0)

#### I a leaving the null values in Violatility because llm needs to understand the there will be null values for starting 20 days for each stock.

In [96]:
merged_df.isnull().sum()

Price
Open                0
High                0
Low                 0
Close               0
Volume              0
Ticker              0
Date                0
Daily Return        0
Log Return          0
MA_20             285
Volatility_20d    300
dtype: int64

Saving as CSV 

In [98]:
merged_df.to_csv("C:/Users/cnaid/Downloads/merged_asset_data.csv", index=False)

### Metadata for better prompt results

In [101]:
metadata = pd.DataFrame([
    ["AAPL", "Tech", "Large Cap", "Stock", "Medium"],
    ["MSFT", "Tech", "Large Cap", "Stock", "Medium"],
    ["AMZN", "Consumer", "Large Cap", "Stock", "Medium"],
    ["JNJ", "Healthcare", "Large Cap", "Stock", "Low"],
    ["NVDA", "Tech", "Large Cap", "Stock", "High"],
    ["PLTR", "Tech", "Mid Cap", "Stock", "High"],
    ["ROKU", "Consumer", "Mid Cap", "Stock", "High"],
    ["ETSY", "Retail", "Mid Cap", "Stock", "High"],
    ["UPST", "Finance", "Small Cap", "Stock", "High"],
    ["SOFI", "Finance", "Small Cap", "Stock", "High"],
    ["QQQ", "Mixed", "-", "ETF", "Medium"],
    ["ARKK", "Mixed", "-", "ETF", "High"],
    ["SPY", "Mixed", "-", "ETF", "Low"],
    ["^GSPC", "Index", "-", "Index", "Low"],
    ["^IXIC", "Index", "-", "Index", "Medium"]
], columns=["Ticker", "Sector", "Market Cap", "Type", "Risk Class"])

metadata.to_csv("C:/Users/cnaid/Downloads/asset_metadata.csv", index=False)
